# Gold Price Prediction

## Importing Libraries

In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
df = pd.read_csv(r"D:\Data Science\Project\Excel File\gld_price_data.csv")

In [3]:
df.head()

,Date,SPX,GLD,USO,SLV,EUR/USD
0,1/2/2008,1447.160034,84.860001,78.470001,15.180,1.471692
1,1/3/2008,1447.160034,85.570000,78.370003,15.285,1.474491
2,1/4/2008,1411.630005,85.129997,77.309998,15.167,1.475492
3,1/7/2008,1416.180054,84.769997,75.500000,15.053,1.468299
4,1/8/2008,1390.189941,86.779999,76.059998,15.590,1.557099


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2290 entries, 0 to 2289
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Date     2290 non-null   object 
 1   SPX      2290 non-null   float64
 2   GLD      2290 non-null   float64
 3   USO      2290 non-null   float64
 4   SLV      2290 non-null   float64
 5   EUR/USD  2290 non-null   float64
dtypes: float64(5), object(1)
memory usage: 107.5+ KB


In [5]:
df.describe()

,SPX,GLD,USO,SLV,EUR/USD
count,2290.000000,2290.000000,2290.000000,2290.000000,2290.000000
mean,1654.315776,122.732875,31.842221,20.084997,1.283653
std,519.111540,23.283346,19.523517,7.092566,0.131547
min,676.530029,70.000000,7.960000,8.850000,1.039047
25%,1239.874969,109.725000,14.380000,15.570000,1.171313
50%,1551.434998,120.580002,33.869999,17.268500,1.303297
75%,2073.010070,132.840004,37.827501,22.882500,1.369971
max,2872.870117,184.589996,117.480003,47.259998,1.598798


In [6]:
df.shape

(2290, 6)

In [7]:
df.isnull().sum()

Date       0
SPX        0
GLD        0
USO        0
SLV        0
EUR/USD    0
dtype: int64

In [8]:
x=df.drop(["Date","GLD"], axis=1)
# whenever you are dropping a column you need to mention axis=1
# whenever you are dropping a row you need to mention axis=0
y=df["GLD"]

In [9]:
print(x)

              SPX        USO      SLV   EUR/USD
0     1447.160034  78.470001  15.1800  1.471692
1     1447.160034  78.370003  15.2850  1.474491
2     1411.630005  77.309998  15.1670  1.475492
3     1416.180054  75.500000  15.0530  1.468299
4     1390.189941  76.059998  15.5900  1.557099
...           ...        ...      ...       ...
2285  2671.919922  14.060000  15.5100  1.186789
2286  2697.790039  14.370000  15.5300  1.184722
2287  2723.070068  14.410000  15.7400  1.191753
2288  2730.129883  14.380000  15.5600  1.193118
2289  2725.780029  14.405800  15.4542  1.182033

[2290 rows x 4 columns]


In [10]:
print(y)

0        84.860001
1        85.570000
2        85.129997
3        84.769997
4        86.779999
           ...    
2285    124.589996
2286    124.330002
2287    125.180000
2288    124.489998
2289    122.543800
Name: GLD, Length: 2290, dtype: float64


## Outliers Analysis

In [11]:
# Removig outliers for gold

q1 = df['GLD'].quantile(0.25)
q3 = df['GLD'].quantile(0.75)
iqr = q3 - q1
q1, q3, iqr

(109.72500025, 132.84000400000002, 23.115003750000028)

In [12]:
upper_limit = q3 + (1.5 * iqr)
lower_limit = q1 - (1.5 * iqr)
lower_limit, upper_limit

(75.05249462499995, 167.51250962500006)

In [13]:
# Find the outliers
df.loc[(df['GLD'] > upper_limit) | (df['GLD'] < lower_limit)]

,Date,SPX,GLD,USO,SLV,EUR/USD
137,9/10/2008,1232.040039,74.220001,82.970001,10.600000,1.399600
138,9/11/2008,1249.050049,73.080002,81.489998,10.320000,1.423994
161,10/22/2008,896.780029,71.709999,54.930000,9.450000,1.294498
162,10/23/2008,908.109985,70.650002,56.599998,9.380000,1.262993
163,10/27/2008,848.919983,72.180000,50.849998,8.850000,1.246463
...,...,...,...,...,...,...
1068,11/19/2012,1386.890015,167.869995,32.669998,32.049999,1.275950
1070,11/21/2012,1391.030029,167.559998,32.119999,32.290001,1.281443
1071,11/23/2012,1409.150024,169.610001,32.320000,32.980000,1.287515
1072,11/26/2012,1406.290039,169.429993,32.189999,33.020000,1.296596


In [14]:
# trim the data FOR GOLD
new_df = df.loc[(df['GLD'] < upper_limit) & (df['GLD'] > lower_limit)]
print('After removing the outliers', len(new_df))
print('Outliers:', len(df)-len(new_df))

After removing the outliers 2175
Outliers: 115


In [15]:
# Removig outliers for USO
q1 = df['USO'].quantile(0.25)
q3 = df['USO'].quantile(0.75)
iqr = q3 - q1
q1, q3, iqr

(14.38, 37.8275015, 23.447501499999994)

In [16]:
upper_limit = q3 + (1.5 * iqr)
lower_limit = q1 - (1.5 * iqr)
lower_limit, upper_limit

(-20.791252249999992, 72.99875374999999)

In [17]:
# find the outliers for uso
df.loc[(df['USO'] > upper_limit) | (df['USO'] < lower_limit)]

,Date,SPX,GLD,USO,SLV,EUR/USD
0,1/2/2008,1447.160034,84.860001,78.470001,15.180,1.471692
1,1/3/2008,1447.160034,85.570000,78.370003,15.285,1.474491
2,1/4/2008,1411.630005,85.129997,77.309998,15.167,1.475492
3,1/7/2008,1416.180054,84.769997,75.500000,15.053,1.468299
4,1/8/2008,1390.189941,86.779999,76.059998,15.590,1.557099
...,...,...,...,...,...,...
146,9/25/2008,1209.180054,86.449997,86.690002,13.030,1.459897
147,9/29/2008,1106.420044,89.570000,77.089996,12.950,1.409642
148,9/30/2008,1166.359985,85.070000,82.010002,11.850,1.401620
149,10/1/2008,1161.060059,85.970001,79.589996,12.330,1.380796


In [18]:
# trim the data FOR USO
new_df = df.loc[(df['USO'] < upper_limit) & (df['USO'] > lower_limit)]
print('After removing the outliers', len(new_df))
print('Outliers:', len(df)-len(new_df))

After removing the outliers 2156
Outliers: 134


In [19]:
# Removig outliers for SLV
q1 = df['SLV'].quantile(0.25)
q3 = df['SLV'].quantile(0.75)
iqr = q3 - q1
q1, q3, iqr

(15.57, 22.8824995, 7.312499500000001)

In [20]:
upper_limit = q3 + (1.5 * iqr)
lower_limit = q1 - (1.5 * iqr)
lower_limit, upper_limit

(4.601250749999998, 33.85124875)

In [21]:
# find the outliers for slv
df.loc[(df['SLV'] > upper_limit) | (df['SLV'] < lower_limit)]

,Date,SPX,GLD,USO,SLV,EUR/USD
694,3/1/2011,1306.329956,140.029999,40.480000,33.869999,1.382151
697,3/4/2011,1321.150024,139.350006,42.330002,34.689999,1.395888
698,3/7/2011,1310.130005,139.720001,42.369999,35.230000,1.399071
699,3/8/2011,1321.819946,139.360001,42.310001,35.180000,1.396394
700,3/9/2011,1320.020020,139.410004,42.049999,35.270000,1.389796
...,...,...,...,...,...,...
913,2/24/2012,1365.739990,172.229996,42.009998,34.369999,1.337363
914,2/27/2012,1367.589966,171.699997,41.180000,34.360001,1.346711
915,2/28/2012,1372.180054,173.490005,40.790001,35.830002,1.340662
917,3/1/2012,1374.089966,166.610001,41.750000,34.439999,1.332090


In [22]:
# trim the data FOR SLV
new_df = df.loc[(df['SLV'] < upper_limit) & (df['SLV'] > lower_limit)]
print('After removing the outliers', len(new_df))
print('Outliers:', len(df)-len(new_df))

After removing the outliers 2173
Outliers: 117


## Splitting into Training and Test Data

In [52]:
x=new_df.drop(["Date","GLD"], axis=1)
# whenever you are dropping a column you need to mention axis=1
# whenever you are dropping a row you need to mention axis=0
y=new_df["GLD"]

In [24]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [25]:
x_train

,SPX,USO,SLV,EUR/USD
1018,1413.489990,36.220001,28.920000,1.253447
1870,2085.449951,11.740000,16.420000,1.132824
118,1245.359985,109.250000,18.545000,1.585590
220,805.219971,28.660000,11.090000,1.288494
2053,2328.949951,11.130000,17.530001,1.061639
...,...,...,...,...
1638,2095.840088,19.660000,15.480000,1.120787
1095,1426.189941,33.369999,29.370001,1.322769
1130,1502.420044,33.430000,27.730000,1.327316
1294,1807.229980,33.189999,18.959999,1.356779


## Feature Scaling

In [26]:
# Featur Scaling
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train,y_train)
x_test_scaled  =scaler.transform(x_test)

## Random Forest

In [27]:
# Random Forest
regressor=RandomForestRegressor(n_estimators=100)

# training the model
regressor.fit(x_train_scaled , y_train)

RandomForestRegressor()

In [28]:
y_pred_RF=regressor.predict(x_test_scaled)

In [29]:
# y_pred_RF

In [70]:
rf_score = metrics.r2_score(y_test,y_pred_RF)
mae = mean_absolute_error(y_test,y_pred_RF)
mse = mean_squared_error(y_test,y_pred_RF)
rmse = np.sqrt(mse)
print("R-squared error of Random forest:", rf_score)
print("mae of Random forest:", mae)
print("mse of Random forest:", mse)
print("rmse of Random forest:", rmse)

R-squared error of Random forest: 0.9901875765574677
mae of Random forest: 1.2569277787336273
mse of Random forest: 5.380150749768362
rmse of Random forest: 2.3195151971410666


## Decision Tree

In [31]:
# Decision tree
dt_regressor = DecisionTreeRegressor()
dt_regressor.fit(x_train_scaled,y_train)


DecisionTreeRegressor()

In [32]:
y_dt_predicted = dt_regressor.predict(x_test_scaled)

In [33]:
# y_dt_predicted

In [73]:
dt_score = metrics.r2_score(y_test,y_dt_predicted)
mae = mean_absolute_error(y_test,y_dt_predicted)
mse = mean_squared_error(y_test,y_dt_predicted)
rmse = np.sqrt(mse)
print("R-squared error of Decision tree:", dt_score)
print("mae of Decision tree:", mae)
print("mse of Decision tree:", mse)
print("rmse of Decision tree:", rmse)

R-squared error of Decision tree: 0.9846872322052614
mae of Decision tree: 1.4496723253275108
mse of Decision tree: 8.395988984208625
rmse of Decision tree: 2.8975833006504965


In [35]:
from sklearn.linear_model import LinearRegression
from scipy import stats

## Linear Regression

In [36]:
model = LinearRegression()
model

LinearRegression()

In [37]:
# x_train_scaled = x_train_scaled.reshape(-1,1)
# x_test_scaled = x_test_scaled.reshape(-1,1)

In [38]:
model.fit(x_train_scaled,y_train)

LinearRegression()

In [39]:
y_pred_linear = model.predict(x_test_scaled)

In [40]:
#y_pred_linear

In [76]:
linear_score = metrics.r2_score(y_test,y_pred_linear)
mae = mean_absolute_error(y_test,y_pred_linear)
mse = mean_squared_error(y_test,y_pred_linear)
rmse = np.sqrt(mse)
print("R-squared error of Linear Regression:", linear_score)
print("mae of Linear Regression:", mae)
print("mse of Linear Regression:", mse)
print("rmse of Linear Regression:", rmse)

R-squared error of Linear Regression: 0.8975640982991402
mae of Linear Regression: 5.695900644269291
mse of Linear Regression: 56.165594215006024
rmse of Linear Regression: 7.494370835167287


## KNN Rrgressor

In [45]:
from sklearn.neighbors import KNeighborsRegressor

In [56]:
knn = KNeighborsRegressor(n_neighbors=3).fit(x_train_scaled, y_train)

In [79]:
y_knn = knn.predict(x_test_scaled)
#y_knn

In [64]:
knn_score = metrics.r2_score(y_test, y_knn)
knn_score

0.9938943533041434

In [77]:
knn_score = metrics.r2_score(y_test,y_knn)
mae = mean_absolute_error(y_test,y_knn)
mse = mean_squared_error(y_test,y_knn)
rmse = np.sqrt(mse)
print("R-squared error of KNN:", knn_score)
print("mae of KNN:", mse)
print("rmse of KNN:", rmse)

R-squared error of KNN: 0.9938943533041434
mae of KNN: 3.3477254463099593
rmse of KNN: 1.8296790555477098


## SVM

In [59]:
from sklearn.svm import SVR

In [65]:
svr = SVR().fit(x_train_scaled,y_train)

In [66]:
y_svr = svr.predict(x_test_scaled)

In [78]:
svr_score = metrics.r2_score(y_test,y_svr)
mae = mean_absolute_error(y_test,y_svr)
mse = mean_squared_error(y_test,y_svr)
rmse = np.sqrt(mse)
print("R-squared error of KNN:", svr_score)
print("mae of KNN:", mse)
print("rmse of KNN:", rmse)

R-squared error of KNN: 0.9425670936809014
mae of KNN: 31.4904565425412
rmse of KNN: 5.611635816991441
